<a href="https://colab.research.google.com/github/cici-zhao/valence-similarity-memory/blob/main/Bert_similarity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Note: Ctrl + F for 'Modify' to see which names need to be replaced for each new run**

# Calculate average cosine similarity of all words on a word list

To use: upload 1 word list

Outputs:
1. csv file with cosine similarity data
2. basic stats about the distribution of cosine similarities (printed to console)

In [ ]:
# !pip install -U transformers

In [ ]:
# load BERT
# Bert base uncased is used
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Get the last layer of embedding of a word/token
def obtain_last_layer_embedding(word_input):
    encoded_input = tokenizer(word_input, return_tensors='pt')
    output = model(**encoded_input)
    return output.last_hidden_state

# Aggregate the embedding of a word/token
def get_cls_embedding(last_hidden_state):
  """
  Extracts the embedding for the [CLS] token.
  """
  # [CLS] token is at index 0
  return last_hidden_state[:, 0, :]

In [ ]:
### Prepare a dataframe to store output

# read csv file that contains all word stimuli & extract words
data = pd.read_csv('Alves_reversed.csv') # Modify: input file name
data_df = pd.DataFrame(data)
words_raw = data_df['Common Words'].tolist()  # Modify: column where words are stored
words = [str(item) for item in words_raw] # Ensure all words are passed as strings

# Create an empty data frame with the specified words as both row and column names to store output
word_table_df = pd.DataFrame(index=words, columns=words)

# Initialize the DataFrame with some values (optional)
# words_df = words_df.fillna(0)


### Loop through the given word list and calculate cosine similarity for all word pairs

# version 1: doesn't carry out comparison if it has been done in reverse order before
# count_row_number = 0
# for current_row, rows in word_table_df.iterrows():
#   count_row_number = count_row_number + 1
#   word_1_embed = get_cls_embedding(obtain_last_layer_embedding(current_row))
#   count_column_number = 0
#   for column in rows.index:
#     count_column_number = count_column_number + 1
#     if (current_row != column) and (count_column_number > count_row_number):
#       # word_table_df.loc[current_row, column] = 'yes'
#       word_2_embed = get_cls_embedding(obtain_last_layer_embedding(column))
#       cos_sim_torch = torch.cosine_similarity(word_1_embed, word_2_embed)
#       word_table_df.loc[current_row, column] = cos_sim_torch
#   print(count_row_number)

# version 2: do all comparisons
count_row_number = 0  # counter to track progress for debugging purposes
for current_row, rows in word_table_df.iterrows():
  count_row_number = count_row_number + 1
  word_1_embed = get_cls_embedding(obtain_last_layer_embedding(current_row))  # get embedding for word on current row
  for column in rows.index:
    if current_row != column: # avoid comparing a word with itself
      word_2_embed = get_cls_embedding(obtain_last_layer_embedding(column)) # get embedding for word on current column
      cos_sim_torch = torch.cosine_similarity(word_1_embed, word_2_embed) # obtain cos sim of the 2 embeddings
      cos_sim_numerical = cos_sim_torch.item()  # convert cos sim to numerical value
      word_table_df.loc[current_row, column] = cos_sim_numerical  # store cos sim value in dataframe
  print(count_row_number)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
### Obtain average cosine similarity for each word

import numpy as np

# word_table = pd.read_csv('embeddings_20_random_neg_4.csv')
# word_table_df = pd.DataFrame(word_table)
# word_table_df = word_table_df.drop('average', axis=1)

# Insert an 'average' column in the dataframe
word_table_df['average'] = np.nan

# Calculate average cosine similarity of each word
for index, row in word_table_df.iterrows():
  word_table_df.loc[index, 'average'] = row.mean()

print(word_table_df)

             kitten      baby   funeral      gift     music     bombs  \
kitten          NaN  0.977461  0.990061  0.990523  0.899364  0.956939   
baby       0.977461       NaN  0.976323  0.979199   0.91911  0.954991   
funeral    0.990061  0.976323       NaN  0.990758   0.90751  0.961929   
gift       0.990523  0.979199  0.990758       NaN    0.9117  0.960722   
music      0.899364   0.91911   0.90751    0.9117       NaN  0.897241   
bombs      0.956939  0.954991  0.961929  0.960722  0.897241       NaN   
death      0.975632  0.970586  0.980676   0.97767  0.921169  0.959156   
hell        0.95689  0.959308  0.957918  0.955336  0.906859   0.94111   
war        0.924473  0.931967  0.936312  0.932745  0.902071  0.944329   
guns       0.963185  0.953973  0.967054  0.965464  0.900489    0.9807   
crime      0.923775    0.9287   0.93537    0.9339  0.898073  0.947099   
hate        0.96246  0.960232  0.965896  0.966224  0.890433  0.959485   
chocolate  0.976869  0.979655  0.976688  0.978609  

In [ ]:
### Polish formatting and write data frame to csv file

# Insert words as the first column for ease of human reading
# word_table_df = word_table_df.drop('Common Words', axis=1)
word_table_df.insert(0, 'Common Words', words)

word_table_df.to_csv('embeddings_Alves_reversed.csv', index=False) # Modify: output file name

In [ ]:
### Obtain statistics about the distribution of cosine similarities

# data1 = pd.read_csv('embeddings_20_random_pos.csv') # Modify: input file name
# averages_df = pd.DataFrame(data1)
averages_df = word_table_df

# Calculate the mean
mean_value = averages_df['average'].mean()

# Calculate the standard deviation
std_dev = averages_df['average'].std()

# Calculate the range (max - min)
data_range = averages_df['average'].max() - averages_df['average'].min()

print(f"Mean: {mean_value}")
print(f"Standard Deviation: {std_dev}")
print(f"Range: {data_range}")


Mean: 0.9512252971529961
Standard Deviation: 0.015188626871965255
Range: 0.05349475542704263


# Analyze if average cosine similarities are different between 2 word lists

To use: upload 2 word lists with cosine similarity data calculated

Output: T-test results of a comparison between average cosine similarities

In [ ]:
from scipy import stats


# read negative word cos sim averages
negatives = pd.read_csv('embeddings_20_random_neg.csv') # Modify: file name
negatives_df = pd.DataFrame(negatives)
negative_avg = negatives_df['average']
print(negative_avg)

# read positive word cos sim averages
positives = pd.read_csv('embeddings_20_random_pos.csv') # Modify: file name
positives_df = pd.DataFrame(positives)
positive_avg = positives_df['average']
print(positive_avg)

t_statistic, p_value = stats.ttest_ind(positive_avg, negative_avg)

alpha = 0.05

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

if p_value < alpha:
  print("The averages are statistically significantly different.")
else:
  print("There is no statistically significant difference between the averages.")

In [ ]:
## Verify data validity:
# run semantic embedding extraction & cosine similarity calculation on 1 word pair,
# ensure result aligns with output entry in data frame

import torch
from sklearn.metrics.pairwise import cosine_similarity

word1 = "funeral"
word2 = "gift"

encoded_input1 = tokenizer(word1, return_tensors='pt')
output1 = model(**encoded_input1)
embedding1 = output1.last_hidden_state[:, 0, :]

encoded_input2 = tokenizer(word2, return_tensors='pt')
output2 = model(**encoded_input2)
embedding2 = output2.last_hidden_state[:, 0, :]

cos_sim_torch = torch.cosine_similarity(embedding1, embedding2)
print(cos_sim_torch)

tensor([0.9908], grad_fn=<SumBackward1>)
